In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
import os
import numpy as np
from hyphen import Hyphenator
hyphe = Hyphenator('en_GB')
import time

pd.set_option('display.max_rows', 255)
pd.set_option('display.max_columns', 100)

In [ ]:
listFossilsGhost = ['FossilKabutops', 'FossilAerodactyl', 'MonGhost',]
listFossilsGhost_Capital = ['FOSSIL_KABUTOPS', 'FOSSIL_AERODACTYL', 'MON_GHOST',]
listFossilsGhost_Names = ['MissingNo.', 'MissingNo.', 'MissingNo.']
listFossilsGhost_Names_Capital = ['MISSINGNO.', 'MISSINGNO.', 'MISSINGNO.']


In [ ]:
def printNCharacters(N, paddingChar=' '):
    stringHere = ''
    for idx in range(N):
        stringHere+=paddingChar
    return stringHere

In [ ]:
def printOneInfoForOneMon(columnOfInterest, PokemonOfInterest, capitalizationMon='Name_allsmall'):
    return DfPkmnInfoFromExcel[columnOfInterest].tolist()[DfPkmnInfoFromExcel[capitalizationMon].tolist().index(PokemonOfInterest)]

def getOneColumn(columnOfInterest):
    return DfPkmnInfoFromExcel[columnOfInterest].tolist()

In [ ]:
def readASMFileReturnLinesAndModifiedPath(asmPath, newSuffix='_fromExcel'):
    asmPathNew = asmPath[:-len('.asm')]+newSuffix+'.asm'

    try:
        file_in = open(asmPath, 'r')
    except:
        linesFromFile = []
    else:
        file_in = open(asmPath, 'r')
        linesFromFile = []
        for y in file_in.readlines():
            linesFromFile.append(y)
        
    return linesFromFile, asmPathNew

In [ ]:
def lineifyDexEntries_OneLine(DexEntryWordByWord=['Hello', 'world'], lengthMax=18, debug=False):
    stringTemp = ''
    lengthTemp = len(stringTemp)
    idxTemp = 0

    keepAddingWords = True
    while keepAddingWords:
        if stringTemp=='':
            stringTempNext = stringTemp + DexEntryWordByWord[idxTemp]
        else:
            stringTempNext = stringTemp + ' ' + DexEntryWordByWord[idxTemp]

        # more than 18 chars
        if len(stringTempNext)>lengthMax:
            # check if just adding one space and two symbols is enough to go beyond limits:
            # previous string must have ended on a word, so a space is needed,
            # also if the word was one-letter long and fit, it would pass the test above,
            # so the best-case scenario is that the new word can be split from the first letter
            if len(stringTemp)+3>lengthMax:
                keepAddingWords = False
            else:
                # check if the next word can be split into syllables
                syllablesTemp = hyphe.syllables(DexEntryWordByWord[idxTemp])
                while syllablesTemp.count('-')>0:
                    syllablesTemp.remove('-')
                print('syllablesTemp =', syllablesTemp) if debug else _
                if len(syllablesTemp)>1:
                    # try add one syllable + space + hyphene at the time
                    for syllableIdx in range(len(syllablesTemp)):
                        syllable = syllablesTemp[syllableIdx]
                        # first syllable is different
                        if syllableIdx==0:
                            stringTempNext = stringTemp + ' ' + syllable + '-'
                        else:
                            stringTempNext = stringTemp[:-1] + syllable + '-'
                        print('stringTempNext =', stringTempNext) if debug else _
                        # check if we went beyond the max allowed length
                        if len(stringTempNext)>lengthMax:
                            keepAddingWords = False
                            break
                        else:
                            # we succeded in adding a syllable to the line
                            stringTemp = stringTempNext
                            syllableIdx+=1
                    #break
                else:
                    # it can't be broken into syllables, henceforth (BEEEH) no hope, end of this line
                    break # keepAddingWords = False ???
        else:
            # all good, let's update the cycle
            stringTemp = stringTempNext
            idxTemp+=1
            # let's check if we ran out of words
            if idxTemp>=len(DexEntryWordByWord):
                wordsLeft = [] # mega ugly but works, a la GF
                break

        try:
            syllableIdx
        except:
            wordsLeft = DexEntryWordByWord[idxTemp:]
        else:
            if syllableIdx==0:
                wordsLeft = DexEntryWordByWord[idxTemp:]
            else:
                syllablesTempToBeGlued = syllablesTemp[syllableIdx:]
                leftoverWord = ''
                for wordPiece in syllablesTempToBeGlued:
                    leftoverWord += wordPiece
                wordsLeft = [leftoverWord] + DexEntryWordByWord[idxTemp+1:]
            print('wordsLeft =', wordsLeft) if debug else _
                
        print('stringTemp =', stringTemp) if debug else _

    return stringTemp, wordsLeft

def lineifyDexEntries(DexEntry='Hello world', lengthMax=18, numberOfLinesMax=6, replacePokemonShort=True, debug=False):
    DexEntry=DexEntry.replace('#MON', 'PoKeMoN') if replacePokemonShort else _
    
    if len(DexEntry)>lengthMax*numberOfLinesMax:
        print('You will not be able to fit your Pokedex entry!')
        print(f'Your Dex entry is {len(DexEntry)} characters long, but with the current settings')
        print(f'you can have max {numberOfLinesMax} lines, each {lengthMax} characters long, for a total of {lengthMax*numberOfLinesMax} characters.')
        print('The constraints above can be changed my inputting different "numberOfLinesMax" and "lengthMax" parameters.')
    else:
        DexEntryWordByWord = DexEntry.rsplit()
        DexEntry_Lineified = []
        while len(DexEntryWordByWord)>0:
            lineHere, DexEntryWordByWord = lineifyDexEntries_OneLine(DexEntryWordByWord, debug=debug)
            print('lineHere =', lineHere) if debug else _
            print('DexEntryWordByWord =', DexEntryWordByWord) if debug else _
            print('-----') if debug else _
            lineHere=lineHere.replace('PoKeMoN', '#MON') if replacePokemonShort else _
            DexEntry_Lineified.append(lineHere)
            
        if len(DexEntry_Lineified)>numberOfLinesMax:
            print('Sorry, I could not split the Dex entry into the required number of lines.')
            print('It may be possible to do so by changing (even against grammar rules) the hyphenation.')
            print('Here is the best I could do:')
            
        return DexEntry_Lineified


In [ ]:
def getFileFolder(filein):
    filePath = ''
    for pathPiece in filein[:-len('.asm')].rsplit(r'/')[:-1]:
        filePath += pathPiece+r'/'
    return filePath

In [ ]:
def tidyOldAndFromExcelFiles(fullNameOfFilesThatNeedsToBe, debug=False):
    oldFileToMove = fullNameOfFilesThatNeedsToBe
#    oldFileToMove = '/Users/Michael/Desktop/Core/Programmi_miei/Pokemon/pretStuff/outputPokemonInfoToExcel/Bands.asm'
    oldFileToMove_newName = oldFileToMove.replace(".asm", "_preExcel.asm")
    print('oldFileToMove =', oldFileToMove) if debug else _
    print('oldFileToMove_newName =', oldFileToMove_newName) if debug else _
    print('getFileFolder(oldFileToMove) =', getFileFolder(oldFileToMove)) if debug else _
    
    newFileToRename = oldFileToMove
    newFileToRename = newFileToRename.replace(".asm", "_fromExcel.asm")
    newFileToRename_newName = newFileToRename.replace("_fromExcel.asm", ".asm")
    print('newFileToRename =', newFileToRename) if debug else _
    print('newFileToRename_newName =', newFileToRename_newName) if debug else _

    #os.system('pwd') if debug else _
    # move in the right folder
    os.chdir(getFileFolder(oldFileToMove))
    os.system('pwd') if debug else _

    # make the dir for the old stuff
    command = 'mkdir preExcel'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # rename the old files with a suffix (_preExcel)
    command = f'mv {oldFileToMove} {oldFileToMove_newName}'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # move said renamed old files to the dir for old stuff
    command = f'mv {oldFileToMove_newName} ./preExcel'
    print(command) if debug else _
    os.system(command)
    time.sleep(1) if debug else _

    # rename the new files (_fromExcel, made from the script) by getting rid of their extension
    command = f'mv {newFileToRename} {newFileToRename_newName}'
    print(command) if debug else _
    os.system(command)


# Read mega file of destiny

In [ ]:
#DfPkmnInfoFromExcel = pd.read_excel('/Users/Michael/Desktop/Core/Programmi_miei/Pokemon/pretStuff/outputPokemonInfoToExcel/testPokemonToExcel.ods', engine='odf')
DfPkmnInfoFromExcel = pd.read_excel('/Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/AllPokemonInfoExcel/PokemonToExcel.ods', engine='odf')

In [ ]:
DfPkmnInfoFromExcel.head()

In [ ]:
allColumns_list = ['Name_Capital', 'Name_allsmall', 'Name_PartlyCapital', 'HPs', 'ATK', 'DEF', 'SPC', 'SPD', 'Type1', 'Type2', 'CatchRate', 'BaseExp', 'Lvl1Moves', 'GrowthRate', 'TMHMMoves', 'EvoInfo', 'LevelUpMoves', 'cries', 'menuIcons', 'palettes', 'species', 'height_dm', 'weight_hg', 'dexTexts', 'ExtraNotes']

In [ ]:
for info in allColumns_list:
    print(f'{info}{printNCharacters(20-len(info))}= {printOneInfoForOneMon(info, "mcharzardx")}')
    

In [ ]:
printOneInfoForOneMon('LevelUpMoves', 'cleffa').rsplit(' ; ')

# preliminary check: are mons' names <= 10 chars? andare names consistent?

In [ ]:
nameColumns_list = ['Name_Capital', 'Name_allsmall', 'Name_PartlyCapital', ]

In [ ]:
DfPkmnInfoFromExcel_Name_Capital = DfPkmnInfoFromExcel['Name_Capital'].tolist()
DfPkmnInfoFromExcel_Name_allsmall = DfPkmnInfoFromExcel['Name_allsmall'].tolist()
DfPkmnInfoFromExcel_Name_PartlyCapital = DfPkmnInfoFromExcel['Name_PartlyCapital'].tolist()

In [ ]:
for idx in range(len(DfPkmnInfoFromExcel)):
    if not ((DfPkmnInfoFromExcel_Name_Capital[idx]==DfPkmnInfoFromExcel_Name_allsmall[idx].upper()) and (DfPkmnInfoFromExcel_Name_Capital[idx]==DfPkmnInfoFromExcel_Name_PartlyCapital[idx].upper())):
        print('Names not equal!')
        print(DfPkmnInfoFromExcel_Name_Capital[idx])
        print(DfPkmnInfoFromExcel_Name_allsmall[idx])
        print(DfPkmnInfoFromExcel_Name_PartlyCapital[idx])
    if len(DfPkmnInfoFromExcel_Name_Capital[idx])>10:
        print('Len > 10!')
    

# pokemon_constants (1/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokemon_constants.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 8

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-12

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')+listFossilsGhost_Capital
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tconst {pkmn}{printNCharacters(19-len(pkmn))}; {idx+1}\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# pokedex_constants (2/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/constants/pokedex_constants.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 5

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-1

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tconst DEX_{pkmn}{printNCharacters(11-len(pkmn))}; {idx+1}\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# base_stats single files (3/13)

In [ ]:
listMonsHere_allsmall = getOneColumn('Name_allsmall')
listMonsHere_Capital = getOneColumn('Name_Capital')
listMonsHere_PartlyCapital = getOneColumn('Name_PartlyCapital')
for pkmnidx in range(len(listMonsHere_allsmall)):
    pkmnHere_allsmall = listMonsHere_allsmall[pkmnidx]
    pkmnHere_Capital = listMonsHere_Capital[pkmnidx]
    pkmnHere_PartlyCapital = listMonsHere_PartlyCapital[pkmnidx]
    
    inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats/'+pkmnHere_allsmall+'.asm'
    linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
    verbose = True

    txtToWrite = open(outputFile, "w")

    txtToWrite.write(f'\tdb DEX_{pkmnHere_Capital} ; pokedex id\n')
    txtToWrite.write(f'\n')

    lineToWrite = '\tdb'
    for statistic in ['HPs', 'ATK', 'DEF', 'SPD', 'SPC']:
        lineToWrite = lineToWrite + printNCharacters(4-len(str(printOneInfoForOneMon(statistic,pkmnHere_allsmall)))) + str(printOneInfoForOneMon(statistic,pkmnHere_allsmall)) + ','
    lineToWrite = lineToWrite[:-1] + '\n' 
    txtToWrite.write(lineToWrite)

    txtToWrite.write(f'\t;   hp  atk  def  spd  spc\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("Type1",pkmnHere_allsmall)}, {printOneInfoForOneMon("Type2",pkmnHere_allsmall)} ; type\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("CatchRate",pkmnHere_allsmall)} ; catch rate\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("BaseExp",pkmnHere_allsmall)} ; base exp\n')
    txtToWrite.write(f'\n')
    txtToWrite.write('\tINCBIN "gfx/pokemon/front/'+pkmnHere_allsmall+'.pic", 0, 1 ; sprite dimensions\n')
    txtToWrite.write(f'\tdw {pkmnHere_PartlyCapital}PicFront, {pkmnHere_PartlyCapital}PicBack\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("Lvl1Moves",pkmnHere_allsmall)} ; level 1 learnset\n')
    txtToWrite.write(f'\tdb {printOneInfoForOneMon("GrowthRate",pkmnHere_allsmall)} ; growth rate\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\t; tm/hm learnset\n')

    try:
        len(printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall))
    except:
        txtToWrite.write('\ttmhm\n')
    else:
        THMLeft = printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall).rsplit()
        while len(THMLeft)>0:
            lineToWrite='\ttmhm ' if len(THMLeft)==len(printOneInfoForOneMon("TMHMMoves",pkmnHere_allsmall).rsplit()) else '\t     '
            lineEnd='\\\n' if len(THMLeft)>5 else '\n'
            for idxTHM in range(min(len(THMLeft),5)):
                lineToWrite = lineToWrite + THMLeft[0] + printNCharacters(14-len(THMLeft[0]))
                THMLeft.pop(0)
            lineToWrite += lineEnd
            txtToWrite.write(lineToWrite)

    txtToWrite.write(f'\t; end\n')
    txtToWrite.write(f'\n')
    txtToWrite.write(f'\tdb BANK({pkmnHere_PartlyCapital}PicFront)\n')
    txtToWrite.write(f'\tassert BANK({pkmnHere_PartlyCapital}PicFront) == BANK({pkmnHere_PartlyCapital}PicBack)\n')

    txtToWrite.close()
    
    tidyOldAndFromExcelFiles(inputFile, debug=False)
    

# base_stats "folder" (4/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/base_stats.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_allsmall')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = 'INCLUDE "data/pokemon/base_stats/'+pkmn+'.asm"\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# evos_moves (5/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/evos_moves.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 4

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
middleLines = ['\tassert_table_length NUM_POKEMON_INDEXES\n', '\n']

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdw {pkmn}EvosMoves\n'
    txtToWrite.write(oneLine)
for crapMon in listFossilsGhost:
    txtToWrite.write(f'\tdw {crapMon}EvosMoves\n')
    
for oneLine in middleLines:
    txtToWrite.write(oneLine)

for pkmn in listMonsHere:   
    txtToWrite.write(f'{pkmn}EvosMoves:\n')
    txtToWrite.write(f'; Evolutions\n')
    
    try:
        len(printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital'))
    except:
        _
    else:
#        if not ' - ' in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital'):
#            txtToWrite.write(f'\tdb {printOneInfoForOneMon("EvoInfo", pkmn, "Name_PartlyCapital")}\n')
#        else:
#            for evolutionOption in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital').rsplit(' - '):
#                txtToWrite.write(f'\tdb {evolutionOption}\n')
        for evolutionOption in printOneInfoForOneMon('EvoInfo', pkmn, 'Name_PartlyCapital').rsplit(' ; '):
            txtToWrite.write(f'\tdb {evolutionOption}\n')
    
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'; Learnset\n')
    
    try:
        len(printOneInfoForOneMon('LevelUpMoves', pkmn, 'Name_PartlyCapital'))
    except:
        _
    else:
        for lvlUpMove in printOneInfoForOneMon('LevelUpMoves', pkmn, 'Name_PartlyCapital').rsplit(' ; '):
            txtToWrite.write(f'\tdb {lvlUpMove.rsplit()[0]}, {lvlUpMove.rsplit()[1]}\n')
    
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'\n')
    
for crapMon in listFossilsGhost:
    txtToWrite.write(f'{crapMon}EvosMoves:\n')
    txtToWrite.write(f'; Evolutions\n') 
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'; Learnset\n')
    txtToWrite.write(f'\tdb 0\n')
    txtToWrite.write(f'\n')
    
txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# names (6/13)

In [ ]:
names_Exceptions_Default = ["NIDORAN_M", "NIDORAN_F", "MR_MIME", "FARFETCHD"]
names_Exceptions_Updated = ["NIDORAN♂", "NIDORAN♀", "MR.MIME", "FARFETCH'D"]

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/names.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    if pkmn in names_Exceptions_Default:
        pkmn = names_Exceptions_Updated[names_Exceptions_Default.index(pkmn)]
    oneLine = '\tdb "'+pkmn+printNCharacters(10-len(pkmn), '@')+'"\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names_Capital:
    oneLine = '\tdb "'+crapMon+printNCharacters(10-len(crapMon), '@')+'"\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# cries (7/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/cries.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 7

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tmon_cry {printOneInfoForOneMon("cries", pkmn, "Name_PartlyCapital")} ; {pkmn}\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names:
    oneLine = f'\tmon_cry SFX_CRY_00, $00, $00 ; {crapMon}\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# menu_icons (8/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/menu_icons.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = '\tnybble '+printOneInfoForOneMon("menuIcons", pkmn, "Name_PartlyCapital")+printNCharacters(15-len(printOneInfoForOneMon("menuIcons", pkmn, "Name_PartlyCapital")))+'; '+pkmn+'\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# palettes (9/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/palettes.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 2

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = '\tdb '+printOneInfoForOneMon("palettes", pkmn, "Name_Capital")+printNCharacters(14-len(printOneInfoForOneMon("palettes", pkmn, "Name_Capital")))+'; '+pkmn+'\n'
    txtToWrite.write(oneLine)

for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# dex_order (10/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_order.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
finalLines_lastIdx = len(linesFromFile)-1
finalLines_firstIdx = finalLines_lastIdx-0

if verbose:
    for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_Capital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdb DEX_{pkmn}\n'
    txtToWrite.write(oneLine)

for crapMon in listFossilsGhost_Names_Capital:
    oneLine = f'\tdb 0 ; {crapMon}\n'
    txtToWrite.write(oneLine)
    
for idx in range(finalLines_firstIdx,finalLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# dex_entries (11/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_entries.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = 0
initialLines_lastIdx = 1

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
middleLines = ['\tassert_table_length NUM_POKEMON_INDEXES\n', '\n', '; string: species name\n', '; height in decimeter\n', '; weight in hectograms\n', '; text entry\n', '\n', ]

In [ ]:
txtToWrite = open(outputFile, "w")

for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
    txtToWrite.write(linesFromFile[idx])
    
listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
    oneLine = f'\tdw {pkmn}DexEntry\n'
    txtToWrite.write(oneLine)
for crapMon in listFossilsGhost_Names:
    txtToWrite.write(f'\tdw {crapMon[:-1]}DexEntry\n')
    
for oneLine in middleLines:
    txtToWrite.write(oneLine)

for pkmn in listMonsHere:   
    txtToWrite.write(f'{pkmn}DexEntry:\n')
    txtToWrite.write('\tdb "'+printOneInfoForOneMon('species', pkmn, 'Name_PartlyCapital')+'@"\n')
    txtToWrite.write('\tdb '+str(printOneInfoForOneMon('height_dm', pkmn, 'Name_PartlyCapital'))+'\n')
    txtToWrite.write('\tdw '+str(printOneInfoForOneMon('weight_hg', pkmn, 'Name_PartlyCapital'))+'\n')
    txtToWrite.write(f'\ttext_far _{pkmn}DexEntry\n')
    txtToWrite.write('\ttext_end\n')
    txtToWrite.write('\n')
    
txtToWrite.write('MissingNoDexEntry:\n')
txtToWrite.write('\tdb "???@"\n') 
txtToWrite.write('\tdb 31\n') 
txtToWrite.write('\tdw 45\n') 
txtToWrite.write('\tdb "コメント\u3000さくせいちゅう@" ; コメント作成中 (Comment to be written)\n') 

txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# dex_text (12/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/data/pokemon/dex_text.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
# cross-checks
# - are all entries theoretically fittable?
for pkmn in getOneColumn('Name_PartlyCapital'):
    if len(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))>18*6:
        print(f'{pkmn} has a too-long entry!')
# - are all entries re-made in 6 lines?
for pkmn in getOneColumn('Name_PartlyCapital'):
    if len(lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital')))!=6:
        print(f'{pkmn} has a '+str(len(lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))))+'-line entry!')

In [ ]:
txtToWrite = open(outputFile, "w")

listMonsHere = getOneColumn('Name_PartlyCapital')
for idx in range(len(listMonsHere)):
    pkmn = listMonsHere[idx]
#    print(pkmn)
#    print(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))
    dexLinesHere = lineifyDexEntries(printOneInfoForOneMon('dexTexts', pkmn, 'Name_PartlyCapital'))
#    print(dexLinesHere)
    txtToWrite.write(f'_{pkmn}DexEntry::\n')
    txtToWrite.write('\ttext "'+dexLinesHere[0]+'"\n')
    txtToWrite.write('\tnext "'+dexLinesHere[1]+'"\n')
    txtToWrite.write('\tnext "'+dexLinesHere[2]+'"\n')
    txtToWrite.write('\n')
    txtToWrite.write('\tpage "'+dexLinesHere[3]+'"\n')
    if len(dexLinesHere)>=5:
        txtToWrite.write('\tnext "'+dexLinesHere[4]+'"\n')
    if len(dexLinesHere)==6:
        txtToWrite.write('\tnext "'+dexLinesHere[5]+'"\n')
    txtToWrite.write('\tdex\n')
    txtToWrite.write('\n')
#    print('---')
    
txtToWrite.close()

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# pics D: (13/13)

In [ ]:
inputFile = '/Users/michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pics.asm'
linesFromFile, outputFile = readASMFileReturnLinesAndModifiedPath(inputFile)
verbose = False

In [ ]:
initialLines_firstIdx = linesFromFile.index('SECTION "Pics 6", ROMX\n')
initialLines_lastIdx = linesFromFile.index('SECTION "Pics 8", ROMX\n')+7

if verbose:
    for idx in range(initialLines_firstIdx,initialLines_lastIdx+1):
        print(repr(linesFromFile[idx]))

In [ ]:
txtToWrite = open(outputFile, "w")

listMonsHere = getOneColumn('Name_PartlyCapital')
listMonsHere_allsmall = getOneColumn('Name_allsmall')

for idx in range(len(listMonsHere)):
    
    if int(idx/30)+1==6:
        txtToWrite.write('\n')
        txtToWrite.write('\n')
#        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+2)+'", ROMX\n')
        txtToWrite.write('SECTION "Pics '+str(6)+'", ROMX\n')
        txtToWrite.write('\n')
        txtToWrite.write('YoungsterPic::     INCBIN "gfx/trainers/youngster.pic"\n')
        txtToWrite.write('BugCatcherPic::    INCBIN "gfx/trainers/bugcatcher.pic"\n')
        txtToWrite.write('LassPic::          INCBIN "gfx/trainers/lass.pic"\n')
        txtToWrite.write('SailorPic::        INCBIN "gfx/trainers/sailor.pic"\n')
        txtToWrite.write('JrTrainerMPic::    INCBIN "gfx/trainers/jr.trainerm.pic"\n')
        txtToWrite.write('JrTrainerFPic::    INCBIN "gfx/trainers/jr.trainerf.pic"\n')
        txtToWrite.write('PokemaniacPic::    INCBIN "gfx/trainers/pokemaniac.pic"\n')
        txtToWrite.write('SuperNerdPic::     INCBIN "gfx/trainers/supernerd.pic"\n')
        txtToWrite.write('HikerPic::         INCBIN "gfx/trainers/hiker.pic"\n')
        txtToWrite.write('BikerPic::         INCBIN "gfx/trainers/biker.pic"\n')
        txtToWrite.write('BurglarPic::       INCBIN "gfx/trainers/burglar.pic"\n')
        txtToWrite.write('EngineerPic::      INCBIN "gfx/trainers/engineer.pic"\n')
        txtToWrite.write('FisherPic::        INCBIN "gfx/trainers/fisher.pic"\n')
        txtToWrite.write('SwimmerPic::       INCBIN "gfx/trainers/swimmer.pic"\n')
        txtToWrite.write('CueBallPic::       INCBIN "gfx/trainers/cueball.pic"\n')
        txtToWrite.write('GamblerPic::       INCBIN "gfx/trainers/gambler.pic"\n')
        txtToWrite.write('BeautyPic::        INCBIN "gfx/trainers/beauty.pic"\n')
        txtToWrite.write('PsychicPic::       INCBIN "gfx/trainers/psychic.pic"\n')
        txtToWrite.write('RockerPic::        INCBIN "gfx/trainers/rocker.pic"\n')
        txtToWrite.write('JugglerPic::       INCBIN "gfx/trainers/juggler.pic"\n')
        txtToWrite.write('TamerPic::         INCBIN "gfx/trainers/tamer.pic"\n')
        txtToWrite.write('BirdKeeperPic::    INCBIN "gfx/trainers/birdkeeper.pic"\n')
        txtToWrite.write('BlackbeltPic::     INCBIN "gfx/trainers/blackbelt.pic"\n')
        txtToWrite.write('Rival1Pic::        INCBIN "gfx/trainers/rival1.pic"\n')
        txtToWrite.write('ProfOakPic::       INCBIN "gfx/trainers/prof.oak.pic"\n')
        txtToWrite.write('ChiefPic::\n')
        txtToWrite.write('ScientistPic::     INCBIN "gfx/trainers/scientist.pic"\n')
        txtToWrite.write('GiovanniPic::      INCBIN "gfx/trainers/giovanni.pic"\n')
        txtToWrite.write('RocketPic::        INCBIN "gfx/trainers/rocket.pic"\n')
        txtToWrite.write('CooltrainerMPic::  INCBIN "gfx/trainers/cooltrainerm.pic"\n')
        txtToWrite.write('CooltrainerFPic::  INCBIN "gfx/trainers/cooltrainerf.pic"\n')
        txtToWrite.write('BrunoPic::         INCBIN "gfx/trainers/bruno.pic"\n')
        txtToWrite.write('BrockPic::         INCBIN "gfx/trainers/brock.pic"\n')
        txtToWrite.write('MistyPic::         INCBIN "gfx/trainers/misty.pic"\n')
        txtToWrite.write('LtSurgePic::       INCBIN "gfx/trainers/lt.surge.pic"\n')
        txtToWrite.write('ErikaPic::         INCBIN "gfx/trainers/erika.pic"\n')
        txtToWrite.write('KogaPic::          INCBIN "gfx/trainers/koga.pic"\n')
        txtToWrite.write('BlainePic::        INCBIN "gfx/trainers/blaine.pic"\n')
        txtToWrite.write('SabrinaPic::       INCBIN "gfx/trainers/sabrina.pic"\n')
        txtToWrite.write('GentlemanPic::     INCBIN "gfx/trainers/gentleman.pic"\n')
        txtToWrite.write('Rival2Pic::        INCBIN "gfx/trainers/rival2.pic"\n')
        txtToWrite.write('Rival3Pic::        INCBIN "gfx/trainers/rival3.pic"\n')
        txtToWrite.write('LoreleiPic::       INCBIN "gfx/trainers/lorelei.pic"\n')
        txtToWrite.write('ChannelerPic::     INCBIN "gfx/trainers/channeler.pic"\n')
        txtToWrite.write('AgathaPic::        INCBIN "gfx/trainers/agatha.pic"\n')
        txtToWrite.write('LancePic::         INCBIN "gfx/trainers/lance.pic"\n')
        txtToWrite.write('JessieJamesPic::   INCBIN "gfx/trainers/jessiejames.pic"\n')
        txtToWrite.write('\n')
        txtToWrite.write('\n')
#        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+3)+'", ROMX\n')
        txtToWrite.write('SECTION "Pics '+str(7)+'", ROMX\n')
        txtToWrite.write('\n')
        txtToWrite.write('RedPicBack::       INCBIN "gfx/player/redb.pic"\n')
        txtToWrite.write('OldManPicBack::    INCBIN "gfx/battle/oldmanb.pic"\n')
        txtToWrite.write('ProfOakPicBack::   INCBIN "gfx/battle/prof.oakb.pic"\n')
        break
        
    if idx%30==0:
        if idx!=0:
            txtToWrite.write('\n')
            txtToWrite.write('\n')
        txtToWrite.write('SECTION "Pics '+str(int(idx/30)+1)+'", ROMX\n')
        txtToWrite.write('\n')
    pkmn = listMonsHere[idx]
    txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx]+'.pic"\n')
    txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx]+'b.pic"\n')
    
for idx2 in range(idx,len(listMonsHere)):
    if idx2%30==0:
        if idx2!=0:
            txtToWrite.write('\n')
            txtToWrite.write('\n')
        txtToWrite.write('SECTION "Pics '+str(int(idx2/30)+3)+'", ROMX\n')
        txtToWrite.write('\n')
    pkmn = listMonsHere[idx2]
    txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx2]+'.pic"\n')
    txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx2]+'b.pic"\n')
        
txtToWrite.write('\n')
txtToWrite.write('FossilKabutopsPic::   INCBIN "gfx/pokemon/front/fossilkabutops.pic"\n')
txtToWrite.write('FossilAerodactylPic::  INCBIN "gfx/pokemon/front/fossilaerodactyl.pic"\n')
txtToWrite.write('GhostPic::             INCBIN "gfx/battle/ghost.pic"\n')

txtToWrite.close()    

In [ ]:
if False:
    txtToWrite = open(outputFile, "w")

    listMonsHere = getOneColumn('Name_PartlyCapital')
    listMonsHere_allsmall = getOneColumn('Name_allsmall')
    for idx in range(len(listMonsHere)):
        if idx%30==0:
            if idx!=0:
                txtToWrite.write('\n')
                txtToWrite.write('\n')
            txtToWrite.write('SECTION "Pics '+str(int(idx/30)+1)+'", ROMX\n')
            txtToWrite.write('\n')
        pkmn = listMonsHere[idx]
        txtToWrite.write(pkmn+'PicFront::'+printNCharacters(22-len(pkmn+'PicFront::'))+'INCBIN "gfx/pokemon/front/'+listMonsHere_allsmall[idx]+'.pic"\n')
        txtToWrite.write(pkmn+'PicBack::'+printNCharacters(22-len(pkmn+'PicBack::'))+'INCBIN "gfx/pokemon/back/'+listMonsHere_allsmall[idx]+'b.pic"\n')

    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+2)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('YoungsterPic::     INCBIN "gfx/trainers/youngster.pic"\n')
    txtToWrite.write('BugCatcherPic::    INCBIN "gfx/trainers/bugcatcher.pic"\n')
    txtToWrite.write('LassPic::          INCBIN "gfx/trainers/lass.pic"\n')
    txtToWrite.write('SailorPic::        INCBIN "gfx/trainers/sailor.pic"\n')
    txtToWrite.write('JrTrainerMPic::    INCBIN "gfx/trainers/jr.trainerm.pic"\n')
    txtToWrite.write('JrTrainerFPic::    INCBIN "gfx/trainers/jr.trainerf.pic"\n')
    txtToWrite.write('PokemaniacPic::    INCBIN "gfx/trainers/pokemaniac.pic"\n')
    txtToWrite.write('SuperNerdPic::     INCBIN "gfx/trainers/supernerd.pic"\n')
    txtToWrite.write('HikerPic::         INCBIN "gfx/trainers/hiker.pic"\n')
    txtToWrite.write('BikerPic::         INCBIN "gfx/trainers/biker.pic"\n')
    txtToWrite.write('BurglarPic::       INCBIN "gfx/trainers/burglar.pic"\n')
    txtToWrite.write('EngineerPic::      INCBIN "gfx/trainers/engineer.pic"\n')
    txtToWrite.write('FisherPic::        INCBIN "gfx/trainers/fisher.pic"\n')
    txtToWrite.write('SwimmerPic::       INCBIN "gfx/trainers/swimmer.pic"\n')
    txtToWrite.write('CueBallPic::       INCBIN "gfx/trainers/cueball.pic"\n')
    txtToWrite.write('GamblerPic::       INCBIN "gfx/trainers/gambler.pic"\n')
    txtToWrite.write('BeautyPic::        INCBIN "gfx/trainers/beauty.pic"\n')
    txtToWrite.write('PsychicPic::       INCBIN "gfx/trainers/psychic.pic"\n')
    txtToWrite.write('RockerPic::        INCBIN "gfx/trainers/rocker.pic"\n')
    txtToWrite.write('JugglerPic::       INCBIN "gfx/trainers/juggler.pic"\n')
    txtToWrite.write('TamerPic::         INCBIN "gfx/trainers/tamer.pic"\n')
    txtToWrite.write('BirdKeeperPic::    INCBIN "gfx/trainers/birdkeeper.pic"\n')
    txtToWrite.write('BlackbeltPic::     INCBIN "gfx/trainers/blackbelt.pic"\n')
    txtToWrite.write('Rival1Pic::        INCBIN "gfx/trainers/rival1.pic"\n')
    txtToWrite.write('ProfOakPic::       INCBIN "gfx/trainers/prof.oak.pic"\n')
    txtToWrite.write('ChiefPic::\n')
    txtToWrite.write('ScientistPic::     INCBIN "gfx/trainers/scientist.pic"\n')
    txtToWrite.write('GiovanniPic::      INCBIN "gfx/trainers/giovanni.pic"\n')
    txtToWrite.write('RocketPic::        INCBIN "gfx/trainers/rocket.pic"\n')
    txtToWrite.write('CooltrainerMPic::  INCBIN "gfx/trainers/cooltrainerm.pic"\n')
    txtToWrite.write('CooltrainerFPic::  INCBIN "gfx/trainers/cooltrainerf.pic"\n')
    txtToWrite.write('BrunoPic::         INCBIN "gfx/trainers/bruno.pic"\n')
    txtToWrite.write('BrockPic::         INCBIN "gfx/trainers/brock.pic"\n')
    txtToWrite.write('MistyPic::         INCBIN "gfx/trainers/misty.pic"\n')
    txtToWrite.write('LtSurgePic::       INCBIN "gfx/trainers/lt.surge.pic"\n')
    txtToWrite.write('ErikaPic::         INCBIN "gfx/trainers/erika.pic"\n')
    txtToWrite.write('KogaPic::          INCBIN "gfx/trainers/koga.pic"\n')
    txtToWrite.write('BlainePic::        INCBIN "gfx/trainers/blaine.pic"\n')
    txtToWrite.write('SabrinaPic::       INCBIN "gfx/trainers/sabrina.pic"\n')
    txtToWrite.write('GentlemanPic::     INCBIN "gfx/trainers/gentleman.pic"\n')
    txtToWrite.write('Rival2Pic::        INCBIN "gfx/trainers/rival2.pic"\n')
    txtToWrite.write('Rival3Pic::        INCBIN "gfx/trainers/rival3.pic"\n')
    txtToWrite.write('LoreleiPic::       INCBIN "gfx/trainers/lorelei.pic"\n')
    txtToWrite.write('ChannelerPic::     INCBIN "gfx/trainers/channeler.pic"\n')
    txtToWrite.write('AgathaPic::        INCBIN "gfx/trainers/agatha.pic"\n')
    txtToWrite.write('LancePic::         INCBIN "gfx/trainers/lance.pic"\n')
    txtToWrite.write('JessieJamesPic::   INCBIN "gfx/trainers/jessiejames.pic"\n')
    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+3)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('RedPicBack::       INCBIN "gfx/player/redb.pic"\n')
    txtToWrite.write('OldManPicBack::    INCBIN "gfx/battle/oldmanb.pic"\n')
    txtToWrite.write('ProfOakPicBack::   INCBIN "gfx/battle/prof.oakb.pic"\n')
    txtToWrite.write('\n')
    txtToWrite.write('\n')
    txtToWrite.write('SECTION "Pics '+str(int(idx/30)+4)+'", ROMX\n')
    txtToWrite.write('\n')
    txtToWrite.write('FossilKabutopsPic::   INCBIN "gfx/pokemon/front/fossilkabutops.pic"\n')
    txtToWrite.write('FossilAerodactylPic::  INCBIN "gfx/pokemon/front/fossilaerodactyl.pic"\n')
    txtToWrite.write('GhostPic::             INCBIN "gfx/battle/ghost.pic"\n')

    txtToWrite.close()    

In [ ]:
tidyOldAndFromExcelFiles(inputFile, debug=False)

# Proxy images just to test

In [ ]:
pkmnWithProxyPics_list = ['mvenusaur', 'mcharzardx', 'mcharzardy', 'mblastoise', 'mbeedrill', 'mpidgeot', 'pichu', 'cleffa' \
                          'igglybuff', ]

In [ ]:
for pkmnToProxy in pkmnWithProxyPics_list:
    print(pkmnToProxy)
    command = f'cp /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/pokeyellow_Venustoise/gfx/pokemon/back/venustoiseb.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/back/{pkmnToProxy}b.png'
    print(command)
    os.system(command)
    command = f'cp /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/pokeyellow_Venustoise/gfx/pokemon/front/venustoise.png /Users/Michael/Desktop/Core/Giochi/Pokémon/DisassemblyYellow/ExtremeYellow/gfx/pokemon/front/{pkmnToProxy}.png'
    print(command)
    os.system(command)
    print('---')

# Sorted THMs

In [ ]:
THMs_sorted = ['MEGA_PUNCH', 'MEGAHORN', 'SWORDS_DANCE', 'COUNTER', 'DRAGON_CLAW', 'TOXIC', 'DARK_PULSE', 'BODY_SLAM', 'METEOR_MASH', 'DOUBLE_EDGE', 'BUBBLEBEAM', 'METRONOME', 'ICE_BEAM', 'BLIZZARD', 'HYPER_BEAM', 'MIRROR_COAT', 'DRAININGKISS', 'FLASH_CANNON', 'SEISMIC_TOSS', 'FEINT_ATTACK', 'GIGA_DRAIN', 'STONE_EDGE', 'WILL_O_WISP', 'THUNDERBOLT', 'THUNDER', 'EARTHQUAKE', 'FISSURE', 'DIG', 'PSYCHIC_M', 'MOONBLAST', 'MIMIC', 'DOUBLE_TEAM', 'REFLECT', 'BIDE', 'HYDRO_PUMP', 'SELFDESTRUCT', 'CLOSE_COMBAT', 'FIRE_BLAST', 'SHADOW_SNEAK', 'OUTRAGE', 'BRAVE_BIRD', 'REST', 'GUNK_SHOT', 'X_SCISSOR', 'THUNDER_WAVE', 'PSYWAVE', 'EXPLOSION', 'ROCK_SLIDE', 'SHADOW_BALL', 'SUBSTITUTE', ]

In [ ]:
len(THMs_sorted)

# Test length of dex entry

In [ ]:
dexEntry_here = "When its anger rose beyond threshold, gained power unfettered by the limits of its physical body"

lineifyDexEntries(dexEntry_here)